In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import pandas as pd
import glob
'''
image_files = glob.glob('./train_folder/ntrain/*')
image_list = []
for file in image_files:
    img = cv2.imread(file, 0)
    img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST)
    img = np.flatten(img)
    image_list.append(img)
'''

"\nimage_files = glob.glob('./train_folder/ntrain/*')\nimage_list = []\nfor file in image_files:\n    img = cv2.imread(file, 0)\n    img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST)\n    img = np.flatten(img)\n    image_list.append(img)\n"

In [20]:
df_train = pd.read_csv('/content/drive/My Drive/SMAI/Assignment-4/q2/train.csv')
print(df_train.head())

       image_file  emotion
0  IMG_10000000fn        1
1   IMG_10000000n        1
2   IMG_10000004f        0
3  IMG_10000004fn        0
4   IMG_10000004n        0


In [0]:
train_files = df_train['image_file']
labels = np.array(df_train['emotion'])
labels = labels.reshape(-1, 1)
test_files = df_test['image_file']

In [22]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
y_train = encode.fit_transform(labels)
print(y_train.shape)

(1941, 5)


In [0]:
X_train = []
for file in train_files:
    img = cv2.imread('/content/drive/My Drive/SMAI/Assignment-4/q2/ntrain/'+file+'.jpg', 0)
    img = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
    img = img.astype('float32')
    # normalize to the range 0-1
    img /= 255.0
    X_train.append(img)

In [0]:
x_train = np.array(X_train)
x_train = x_train.reshape(len(x_train),128, 128,1)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (128,128,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [38]:
model.fit(x_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
1941/1941 [==============================] - 130s 67ms/step - loss: 1.2758 - accuracy: 0.4735
Epoch 2/50
1941/1941 [==============================] - 132s 68ms/step - loss: 1.1674 - accuracy: 0.5487
Epoch 3/50
1941/1941 [==============================] - 130s 67ms/step - loss: 0.9930 - accuracy: 0.6244
Epoch 4/50
1941/1941 [==============================] - 130s 67ms/step - loss: 0.7995 - accuracy: 0.7038
Epoch 5/50
1941/1941 [==============================] - 129s 66ms/step - loss: 0.6751 - accuracy: 0.7501
Epoch 6/50
1941/1941 [==============================] - 131s 68ms/step - loss: 0.4942 - accuracy: 0.8166
Epoch 7/50
1941/1941 [==============================] - 129s 66ms/step - loss: 0.4461 - accuracy: 0.8382
Epoch 8/50
1941/1941 [==============================] - 129s 66ms/step - loss: 0.3047 - accuracy: 0.8867
Epoch 9/50
1941/1941 [==============================] - 129s 66ms/step - loss: 0.2907 - accuracy: 0.8934
Epoch 10/50
1941/1941 [==============================] 

In [40]:
df_test = pd.read_csv('/content/drive/My Drive/SMAI/Assignment-4/q2/test.csv')
print(df_test.head())
test_files = df_test['image_file']

      image_file
0  IMG_10000000f
1  IMG_10000013f
2  IMG_10000018f
3  IMG_10000030f
4  IMG_10000032f


In [0]:
X_test = []
for file in test_files:
    img = cv2.imread('/content/drive/My Drive/SMAI/Assignment-4/q2/ntest/'+file+'.jpg', 0)
    img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    img = img.astype('float32')
    # normalize to the range 0-1
    img /= 255.0
    X_test.append(img)

In [0]:
x_test = np.array(X_test) 
x_test = x_test.reshape(len(x_test),256, 256,1)

In [0]:
output = model.predict(x_test)

In [0]:
result=[]
for i in range(len(output)):
    result.append(np.argmax(output[i]))
np.savetxt("/content/drive/My Drive//SMAI/Assignment-4/q2/submission.csv", result,fmt="%d",header ="emotion",comments='', delimiter=",")